In [2]:
import csv
import time
import re
import string
import requests
import pytz
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from email.utils import parsedate_tz
from email.utils import mktime_tz
from sqlalchemy import Column, Integer, DateTime, PickleType, String
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [24]:
# Copyright (c) 2018 Sergio Lira <sergio.lira@gmail.com>
#
# MIT Licence. See http://opensource.org/licenses/MIT
#
# Created on 2018-12-03
#

engine = create_engine('sqlite://', echo=False)
Base = declarative_base(bind=engine)

class TimeEntry(Base):
    __tablename__ = 'time_entry'
    id = Column(Integer, primary_key=True)
    date_time = Column(DateTime)
    sdate = Column(String)
    month = Column(Integer)
    day = Column(Integer)
    year = Column(Integer)
    concepts = Column(PickleType)   
        
    def __init__(self, date_time, sdate, month, day, year, concepts):
        self.date_time = date_time
        self.sdate = sdate
        self.month = month
        self.day = day
        self.year = year
        self.concepts = concepts
        
class TimeSpent(Base):
    __tablename__ = 'time_spent'
    id = Column(Integer, primary_key=True)
    sdate = Column(String)
    year = Column(Integer)
    month = Column(Integer)
    day = Column(Integer)
    concept = Column(String)
    
    def __init__(self, sdate, month, day, year, concept):
        self.sdate = sdate
        self.month = month
        self.day = day
        self.year = year
        self.concept = concept

Base.metadata.create_all()
Session = sessionmaker(bind=engine)

class TimeTracker:
    
    def __init__(self, content, n_days=15, n_months = 3):
        self._content = content
        self.n_days = n_days
        self.n_months = n_months
        self._db = Session()

    def load_content(self):
        """
        Initializes the TimeTracker database 
        If the content is of file type txt it initializes by parsing the file else by reading each row in content.
        returns row count added
        """
        #Set look back date and cut_date from max between n_days and n_months        
        look_back = max(self.n_days//31, self.n_months)
        self._cut_date = datetime.today() - relativedelta(months=look_back)
        self._cut_date = date(self._cut_date.date().year, self._cut_date.date().month, 1)
        
        if self._content.endswith('.txt'):
            return self._load_fromm_file()
        else:
            return self._load_fromm_file()
        
        
    def _extract_values_from_row(self, row):
        #Clean date row, format and extract date fields
        #row[0] = row[0].replace(",","").replace("at","")
        date_time= datetime.fromtimestamp(mktime_tz(parsedate_tz(row[0])))
        date_time = date_time.astimezone(pytz.utc)
        sdate = date_time.strftime("%Y-%m-%d")
        month = int(date_time.strftime('%m'))
        day = int(date_time.strftime('%d'))
        year = int(date_time.strftime('%Y'))
        #Extract question and score
        concepts = row[1].strip().split(';')      
        return TimeEntry(date_time, sdate, month, day, year, concepts)
        
    def _load_fromm_file(self):
        with open(self._content, "r") as txtfile:        
            reader = csv.reader(txtfile, delimiter='|', quotechar='"')    
            row_count = 0
            insert_count = 0
            for row in reader:
                #Skip any row that does not matcht he format, including empty rows.
                if len(row) == 2:
                    #Clean date row, format and extract date fields
                    time_entry = self._extract_values_from_row(row)
                    #Only insert date if it will be used
                    if date(time_entry.year, time_entry.month, time_entry.day) >= self._cut_date:
                        #Writte to in-memory database            
                        self._db.add(time_entry)
                        self._db.commit()
                        insert_count += 1 
                    row_count += 1
        return row_count, insert_count
    
    def _calculate_time(self):
        entries = [entry for entry in self._db.query(TimeEntry)]
        if len(entries) > 0 :
            last_entry = entries[0]
            for current_entry in entries[1:]:
                time_delta = current_entry.date_time - last_entry.date_time
                for concept in last_entry.concepts:
                    print()
                    #time_spent = TimeSpent(sdate, month, day, year, concept, time_delta)
                last_entry = current_entry

In [25]:
log_file = '/Users/sergio/Dropbox/Reference/Automation/LogTime.txt'
tt = TimeTracker(log_file)
tt.load_content()
tt._calculate_time()

#for entry in tt._db.query(TimeEntry):
   #print(entry.id, entry.dateTime, entry.sdate, entry.month, entry.day, entry.year, entry.concepts)

['Entertainment'] 0:56:35 : 2018-11-24 03:08:45 - 2018-11-24 04:05:20
['Chores'] 0:03:55 : 2018-11-24 04:05:20 - 2018-11-24 04:09:15
['Entertainment'] 0:50:25 : 2018-11-24 04:09:15 - 2018-11-24 04:59:40
['Exercising'] 0:12:56 : 2018-11-24 04:59:40 - 2018-11-24 05:12:36
['Context Switch'] 0:00:51 : 2018-11-24 05:12:36 - 2018-11-24 05:13:27
['Administrative'] 0:25:35 : 2018-11-24 05:13:27 - 2018-11-24 05:39:02
['Chores'] 0:10:00 : 2018-11-24 05:39:02 - 2018-11-24 05:49:02
['Entertainment'] 0:49:45 : 2018-11-24 05:49:02 - 2018-11-24 06:38:47
['Chores'] 0:08:51 : 2018-11-24 06:38:47 - 2018-11-24 06:47:38
['Entertainment'] 1:22:55 : 2018-11-24 06:47:38 - 2018-11-24 08:10:33
['Sleeping'] 8:53:01 : 2018-11-24 08:10:33 - 2018-11-24 17:03:34
['Self-Care'] 1:58:58 : 2018-11-24 17:03:34 - 2018-11-24 19:02:32
['Administrative'] 0:36:03 : 2018-11-24 19:02:32 - 2018-11-24 19:38:35
['Chores'] 0:42:44 : 2018-11-24 19:38:35 - 2018-11-24 20:21:19
['Commuting'] 0:04:35 : 2018-11-24 20:21:19 - 2018-11-24 

https://www.digitalocean.com/community/tutorials/data-analysis-and-visualization-with-pandas-and-jupyter-notebook-in-python-3
http://pbpython.com/simple-graphing-pandas.html